<hr>
# 1. Intro to sequential models API
<hr>
## 1.1. Model declaration

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
#sequential model instantiation using class constructor
model = Sequential(layers=[Dense(32, input_shape=(784,)), Activation('relu'), Dense(10), Activation('softmax')])

In [ ]:
#same as before but explicit
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

<hr>
## 1.2. Model compilation

In [ ]:
#model compilation

#multi-class classification problem
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#binary classification problem
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

#regression problem
model.compile(optimizer='rmsprop', loss='mse')

### 1.2.1. Custom metrics ans loss functions
Some problems require custom metrics and/or loss functions

In [ ]:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def myloss(y_true, y_pred):
    return K.sqrt(K.mean(y_true - y_pred)**2)

model.compile(optimizer='rmsprop', loss=myloss , metrics=[mean_pred])

<hr>
# 1.3. Model training
## 1.3.1. Binary model

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

#generate dummy data with scikit learn
#import numpy as np
#data = np.random.random((1000,100))
#labels = np.random.randint(2, size=(1000, 1))

from sklearn.datasets import make_classification

dataset = make_classification(n_samples = 1000, n_features = 100, n_classes = 2, class_sep=0.9)
data = dataset[0]
labels = dataset[1]

#train the model iterating on the data on batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

<hr>
## 1.3.2. Multiclass model

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

from sklearn.datasets import make_classification

#generate dummy data with scikit learn
dataset = make_classification(n_samples = 1000, n_features = 100, n_informative=5, n_classes = 10, class_sep=0.9, n_clusters_per_class=1)
data = dataset[0]
labels = dataset[1]

#convert labels to one-hot-encoding representation
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

#train
model.fit(data, one_hot_labels, epochs=20, batch_size=32)

<hr>
# 2. Sequential complex models

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

<h>
## 2.1. Multilayer Perceptron for multi-class softmax classification

In [ ]:
#DATASET
from sklearn.datasets import make_classification

dataset = make_classification(n_samples = 1000, n_features = 20, n_informative=10, n_classes = 3, class_sep=0.98, n_clusters_per_class=1)
data = dataset[0]
labels = dataset[1]

one_hot_labels = keras.utils.to_categorical(labels, num_classes=3)

#train and test datasets 70-30
random_index = np.random.permutation(1000)

train_data = data[random_index[range(900)]]
train_labels = one_hot_labels[random_index[range(900)]]
test_data = data[random_index[range(900,1000)]]
test_labels = one_hot_labels[random_index[range(900, 1000)]]

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs = 100, batch_size=128)
print("Train finished")

score = model.evaluate(test_data, test_labels, batch_size=128)

print score

## 2.2. Multilayer perceptron for binary classification

In [ ]:
#DATASET

dataset = make_classification(n_samples = 1000, n_features = 20, n_informative=2, n_classes = 2, class_sep=0.98, n_clusters_per_class=1)
data = dataset[0]
labels = dataset[1]

#train and test datasets 70-30
random_index = np.random.permutation(1000)

train_data = data[random_index[range(900)]]
train_labels = labels[random_index[range(900)]]
test_data = data[random_index[range(900,1000)]]
test_labels = labels[random_index[range(900, 1000)]]

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.01, momentum=0.9, decay=1e-6)

model.compile(optimizer=sgd, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, batch_size=128)

print(model.evaluate(test_data, test_labels, batch_size=128))

<hr>
## 2.3. Convolutional network (like VGG) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [ ]:
#DATASET
# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

print score

### CIFAR / MNIST dataset example

In [ ]:
from keras import utils
from keras.datasets import cifar10
from keras.datasets import mnist

batch_size = 32
num_classes = 10
epochs = 100

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

train_index = np.random.permutation(1000)
test_index = np.random.permutation(100)

x_train = x_train.astype('float32')[train_index]
x_test = x_test.astype('float32')[test_index]
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)[train_index]
y_test = utils.to_categorical(y_test, num_classes)[test_index]

In [ ]:
print x_train.shape

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=x_train.shape[1:], padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

print score

<hr>
## 2.4. Sequence classification with LSTM ==> case: imdb sentiment analysis

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

from keras.datasets import imdb

from keras.preprocessing import sequence

In [ ]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

#load imdb dataset (sentiment analysis)
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

In [ ]:
#pad sequences ==> [sequence, output]
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=15, batch_size=batch_size, validation_data=(x_test, y_test))

In [ ]:
#save model
model_json = model.to_json()
with open("imdb_sentiment_model.json", "w") as json_file:
    json_file.write(model_json)

#save weights
model.save_weights("imdb_sentiment_model_weights.h5")

In [ ]:
word_to_id = keras.datasets.imdb.get_word_index()

In [34]:
print word_to_id['spiders']

16115


<hr>
## 2.5. Sequence classification with 1D Convolution ==> case: imdb sentiment analysis

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [ ]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [ ]:
#load imdb dataset (sentiment analysis)
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

In [ ]:
#pad sequences ==> [sequence, output]
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
model = Sequential()
#embedding layer
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))
#convolutional layer
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
model.add(GlobalAveragePooling1D())
#MLP layer
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dropout(0.2))
#Sigmoid layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_data=(x_test, y_test))

<hr>
## 2.6. Sequence classification with stacked LSTM==> case: imdb sentiment analysis

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

from keras.datasets import imdb

from keras.preprocessing import sequence

Using TensorFlow backend.


In [13]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

#load imdb dataset (sentiment analysis)
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#pad sequences ==> [sequence, output]
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Loading data...
(25000, 'train sequences')
(25000, 'test sequences')
Pad sequences (samples x time)


In [9]:
model = Sequential()

model.add(Embedding(max_features, 128))

model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

import time

start = time.time()
model.fit(x_train, y_train, epochs=1, batch_size=batch_size, validation_data=(x_test, y_test))
end = time.time()

print(end-time)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 140s 6ms/step - loss: 0.4340 - acc: 0.7922 - val_loss: 0.3536 - val_acc: 0.8439
142.674567938


<hr>
## 2.7. Sequence classification with bidirectional LSTM==> case: imdb sentiment analysis

In [14]:
from keras.layers import Bidirectional

In [15]:
model = Sequential()

model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

import time

start = time.time()
model.fit(x_train, y_train, epochs=1, batch_size=batch_size, validation_data=(x_test, y_test))
end = time.time()

print(end-start)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 105s 4ms/step - loss: 0.4220 - acc: 0.8022 - val_loss: 0.3587 - val_acc: 0.8412
106.84355092
